In [1]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens1MReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data("/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/data") #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

ModuleNotFoundError: No module named 'Data_manager'

In [3]:
import optuna
import numpy as np
import pandas as pd
import os

def create_objective_d(recommender_instance, URM_train, URM_validation, hyperparam_space, evaluator_validation, cutoff, directory_path):
    def objective(trial):
        # Gather and set hyperparameters
        hyperparams = {name: trial.suggest_categorical(name, space) for name, space in hyperparam_space.items()}

        # Initialize and train the recommender
        
        recommender_instance.fit(**hyperparams)

        # Evaluate recommender
        user_id_array = np.arange(URM_validation.shape[0])
        
        result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
        result_df['hyperparams'] = str(hyperparams)

        # Assuming df_row is the DataFrame containing the row to append

        filename = directory_path + recommender_instance.RECOMMENDER_NAME + ".csv"

        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)
    
        return result_df.loc[cutoff]['NDCG']

    return objective


In [4]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

from Diffusion.DiffusionRecommender import DiffusionAutoencoderRecommender_OptimizerMask
import torch



In [11]:
itemKNN_instance2 = ItemKNNCFRecommender(URM_train=URM_train)
itemKNN_instance2.fit(topK=96, shrink =59, similarity ='cosine', normalize=True, feature_weighting='TF-IDF')
result_df, _ = evaluator_validation.evaluateRecommender(itemKNN_instance2)
result_df

ItemKNNCFRecommender: URM Detected 211 ( 5.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 3990.75 column/sec. Elapsed time 0.97 sec
EvaluatorHoldout: Processed 5961 (100.0%) in 3.91 sec. Users per second: 1523


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.172068,0.225724,0.152794,0.090517,0.114132,0.392586,0.1897,0.16186,0.744674,0.584341,...,0.986921,0.734934,0.986921,0.064196,8.296197,0.995206,0.183438,0.76718,1.686127,0.066146
50,0.101315,0.394449,0.388508,0.034652,0.112369,0.40243,0.268621,0.160718,0.925851,0.729563,...,0.986921,0.913742,0.986921,0.115281,9.205504,0.998123,0.32941,0.851267,1.367718,0.342828


In [5]:
#load validation(best_model) models instances

itemKNN_instance = ItemKNNCFRecommender(URM_train=URM_train)
itemKNN_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/ItemKNNCFRecommender_cosine_best_model")

slim_elastic_instance = SLIMElasticNetRecommender(URM_train=URM_train)
slim_elastic_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/SLIMElasticNetRecommender_best_model")

#matrix_fact_instance = PureSVDRecommender(URM_train=URM_train)
#matrix_fact_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/PureSVDRecommender_best_model")

matrix_fact_instance = IALSRecommender(URM_train=URM_train)
matrix_fact_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/IALSRecommender_best_model")


auto_diffusion_instance = DiffusionAutoencoderRecommender_OptimizerMask(URM_train=URM_train, use_gpu=False)
auto_diffusion_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/DiffusionAutoencoderRecommender_best_model")


ItemKNNCFRecommender: URM Detected 211 ( 5.4%) items with no interactions.
ItemKNNCFRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/ItemKNNCFRecommender_cosine_best_model'
ItemKNNCFRecommender: Loading complete
SLIMElasticNetRecommender: URM Detected 211 ( 5.4%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/SLIMElasticNetRecommender_best_model'
SLIMElasticNetRecommender: Loading complete
IALSRecommender: URM Detected 211 ( 5.4%) items with no interactions.
IALSRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/IALSRecommender_best_model'
IALSRecommender: Loading complete
DiffusionAutoencoderRecommender: URM Detected 211 ( 5.4%) items with no interactions.
DiffusionAutoencoderRecommender: Loading model from 

In [6]:

itemKNN_instance_test = ItemKNNCFRecommender(URM_train=URM_train+URM_validation)
itemKNN_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/ItemKNNCFRecommender_cosine_best_model_last")

slim_elastic_instance_test = SLIMElasticNetRecommender(URM_train=URM_train)
slim_elastic_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/SLIMElasticNetRecommender_best_model_last")

pure_svd_instance_test = PureSVDRecommender(URM_train=URM_train)
pure_svd_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/PureSVDRecommender_best_model_last")


matrix_fact_instance_test = IALSRecommender(URM_train=URM_train)
matrix_fact_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/IALSRecommender_best_model_last")


auto_diffusion_instance_test = DiffusionAutoencoderRecommender_OptimizerMask(URM_train=URM_train, use_gpu=False)
auto_diffusion_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/DiffusionAutoencoderRecommender_best_model_last")


ItemKNNCFRecommender: URM Detected 194 ( 5.0%) items with no interactions.
ItemKNNCFRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/ItemKNNCFRecommender_cosine_best_model_last'
ItemKNNCFRecommender: Loading complete
SLIMElasticNetRecommender: URM Detected 211 ( 5.4%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/SLIMElasticNetRecommender_best_model_last'
SLIMElasticNetRecommender: Loading complete
PureSVDRecommender: URM Detected 211 ( 5.4%) items with no interactions.
PureSVDRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/PureSVDRecommender_best_model_last'
PureSVDRecommender: Loading complete
IALSRecommender: URM Detected 211 ( 5.4%) items with no interactions.
IALSRecommender: Loading model from file 

In [11]:
result_df, _ = evaluator_validation.evaluateRecommender(matrix_fact_instance)
result_df

EvaluatorHoldout: Processed 5961 (100.0%) in 3.62 sec. Users per second: 1648


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.186345,0.245822,0.169608,0.09839,0.124377,0.414177,0.20549,0.177583,0.780909,0.626442,...,0.986921,0.770695,0.986921,0.073846,8.557885,0.996611,0.211011,0.79138,1.595006,0.066525
50,0.111149,0.434059,0.427869,0.03877,0.127246,0.423121,0.293169,0.176459,0.93944,0.786603,...,0.986921,0.927152,0.986921,0.138295,9.492902,0.998854,0.39517,0.877844,1.238788,0.347542


In [7]:
from RecommenderLuca import TwoLevelRec
import importlib

# Make some changes to your_module.py

importlib.reload(TwoLevelRec)
from RecommenderLuca.TwoLevelRec import TwoLevelRec

In [16]:
recommenders_instances = {itemKNN_instance, slim_elastic_instance, matrix_fact_instance}
recommenders_instances_test = {itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test}
for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
    print(traditional_rec.RECOMMENDER_NAME)
    print(traditional_rec_test.RECOMMENDER_NAME)

IALSRecommender
ItemKNNCFRecommender
ItemKNNCFRecommender
SLIMElasticNetRecommender
SLIMElasticNetRecommender
IALSRecommender


In [14]:
from datetime import datetime
from RecommenderLuca.TwoLevelRec import TwoLevelRec

inference_timesteps = {1,5,25,50}
recommenders_instances = {itemKNN_instance, slim_elastic_instance, matrix_fact_instance}
recommenders_instances_test = {itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test}

print(recommenders_instance for recommenders_instance in recommenders_instances)

today_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


for inference_timestep in inference_timesteps:
    auto_diffusion_instance.inference_timesteps = inference_timestep
    auto_diffusion_instance_test.inference_timesteps = inference_timestep
    two_level_rec = TwoLevelRec(URM_train=URM_train, recommender1= auto_diffusion_instance, recommender2=itemKNN_instance, max_cutoff=max(cutoff_list))
    for cutoff in cutoff_list:
        # Define the hyperparameter space
        two_level_rec_hyperparam_space = {
            "n_items_to_rank": list(range(cutoff, 300+cutoff)),
        }
        for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
            two_level_rec.set_rec2(traditional_rec)
             # Create the directory with today's date
            directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
            os.makedirs(directory_path, exist_ok=True)
            
            print(traditional_rec.RECOMMENDER_NAME)
            
            print(two_level_rec.RECOMMENDER_NAME)

           

            # Create an objective function
            objective = create_objective_d(recommender_instance=two_level_rec, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_level_rec_hyperparam_space, evaluator_validation=evaluator_validation, cutoff = cutoff, directory_path=directory_path)

            # Create the study and run optimization
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=50,show_progress_bar=True)
            
            two_level_rec1 = TwoLevelRec(URM_train=URM_train+URM_validation, recommender1= auto_diffusion_instance_test, recommender2=traditional_rec_test, max_cutoff=max(cutoff_list))
            two_level_rec1.fit(** study.best_params)
            filename = "best_models_last_" + str(inference_timestep) + "_timesteps" + ".csv"
            filename = os.path.join(directory_path, filename)
            filename2 = auto_diffusion_instance.RECOMMENDER_NAME + "_" + traditional_rec.RECOMMENDER_NAME + "_" + str(inference_timestep)
            filename2 = os.path.join(directory_path, filename2)
            result_df, _ = evaluator_test.evaluateRecommender(two_level_rec1)
            result_df["RECOMMENDER"] = two_level_rec1.RECOMMENDER_NAME
            result_df["PARAMS"] = str(study.best_params)

            
            # Check if file exists
            if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
            else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        

<generator object <genexpr> at 0x7f78ccf02dd0>
Recommender_Base_Class: URM Detected 211 ( 5.4%) items with no interactions.


[I 2023-07-31 03:41:55,253] A new study created in memory with name: no-name-a76fb45c-a600-4c94-ab41-eb1149f47109
  0%|          | 0/50 [00:00<?, ?it/s]

SLIMElasticNetRecommender
DiffusionAutoencoderRecommender_SLIMElasticNetRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5961 (100.0%) in 4.28 sec. Users per second: 1391


Best trial: 0. Best value: 0.224704:   2%|▏         | 1/50 [00:04<03:31,  4.31s/it]

[I 2023-07-31 03:41:59,563] Trial 0 finished with value: 0.22470394825909362 and parameters: {'n_items_to_rank': 260}. Best is trial 0 with value: 0.22470394825909362.
EvaluatorHoldout: Processed 5961 (100.0%) in 4.16 sec. Users per second: 1431


...

Best trial: 0. Best value: 0.240484:  98%|█████████▊| 49/50 [03:03<00:03,  3.77s/it]

[I 2023-07-31 05:18:40,956] Trial 48 finished with value: 0.24048367348205787 and parameters: {'n_items_to_rank': 286}. Best is trial 0 with value: 0.24048367348205787.
EvaluatorHoldout: Processed 5961 (100.0%) in 3.75 sec. Users per second: 1589


Best trial: 0. Best value: 0.240484: 100%|██████████| 50/50 [03:07<00:00,  3.75s/it]

[I 2023-07-31 05:18:44,739] Trial 49 finished with value: 0.24048367348205787 and parameters: {'n_items_to_rank': 340}. Best is trial 0 with value: 0.24048367348205787.
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.


EvaluatorHoldout: Processed 5976 (100.0%) in 3.94 sec. Users per second: 1516


In [16]:
from datetime import datetime
from RecommenderLuca.TwoLevelRec import TwoLevelRec

inference_timesteps = {1,5,25,50}
recommenders_instances = {itemKNN_instance, slim_elastic_instance, matrix_fact_instance}
recommenders_instances_test = {itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test}

print(recommenders_instance for recommenders_instance in recommenders_instances)

today_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


for inference_timestep in inference_timesteps:
    auto_diffusion_instance.inference_timesteps = inference_timestep
    for cutoff in cutoff_list:
        # Define the hyperparameter space
        two_level_rec_hyperparam_space = {
            "n_items_to_rank": list(range(cutoff, 300+cutoff)),
        }
        for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
            two_level_rec = TwoLevelRec(URM_train=URM_train, recommender1= traditional_rec, recommender2=auto_diffusion_instance, max_cutoff=max(cutoff_list))
             # Create the directory with today's date
            directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
            os.makedirs(directory_path, exist_ok=True)
            
            print(traditional_rec.RECOMMENDER_NAME)
            
            print(two_level_rec.RECOMMENDER_NAME)

           

            # Create an objective function
            objective = create_objective_d(recommender_instance=two_level_rec, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_level_rec_hyperparam_space, evaluator_validation=evaluator_validation, cutoff = cutoff, directory_path=directory_path)

            # Create the study and run optimization
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=50,show_progress_bar=True)
            
            two_level_rec_test = TwoLevelRec(URM_train=URM_train+URM_validation, recommender1= traditional_rec_test, recommender2=auto_diffusion_instance_test, max_cutoff=max(cutoff_list))
            two_level_rec_test.fit(** study.best_params)
            filename = "best_models_last_" + str(inference_timestep) + "_timesteps" + ".csv"
            filename = os.path.join(directory_path, filename)
            filename2 = auto_diffusion_instance.RECOMMENDER_NAME + "_" + traditional_rec.RECOMMENDER_NAME + "_" + str(inference_timestep)
            filename2 = os.path.join(directory_path, filename2)
            result_df, _ = evaluator_test.evaluateRecommender(two_level_rec_test)
            result_df["RECOMMENDER"] = two_level_rec_test.RECOMMENDER_NAME
            result_df["PARAMS"] = str(study.best_params)
            print(two_level_rec.RECOMMENDER_NAME)
            print(two_level_rec_test.RECOMMENDER_NAME)


            
            # Check if file exists
            if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
            else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        

<generator object <genexpr> at 0x7f78ebbe6660>
Recommender_Base_Class: URM Detected 211 ( 5.4%) items with no interactions.


[I 2023-07-31 08:40:58,847] A new study created in memory with name: no-name-c0f50e73-1c0b-443f-9ce3-de86f5c402fa
  0%|          | 0/50 [00:00<?, ?it/s]

SLIMElasticNetRecommender
SLIMElasticNetRecommender_DiffusionAutoencoderRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5961 (100.0%) in 4.98 sec. Users per second: 1197


Best trial: 0. Best value: 0.218367:   2%|▏         | 1/50 [00:05<04:05,  5.00s/it]

[I 2023-07-31 08:41:03,849] Trial 0 finished with value: 0.21836671141447161 and parameters: {'n_items_to_rank': 285}. Best is trial 0 with value: 0.21836671141447161.
EvaluatorHoldout: Processed 5961 (100.0%) in 4.88 sec. Users per second: 1222


...

[I 2023-07-31 08:54:57,965] Trial 4 finished with value: 0.30603507573342675 and parameters: {'n_items_to_rank': 171}. Best is trial 1 with value: 0.30637734174377584.
EvaluatorHoldout: Processed 5961 (100.0%) in 5.16 sec. Users per second: 1154


Best trial: 1. Best value: 0.306377:  12%|█▏        | 6/50 [00:29<03:39,  4.99s/it]

[I 2023-07-31 08:55:03,148] Trial 5 finished with value: 0.30588527421385225 and parameters: {'n_items_to_rank': 326}. Best is trial 1 with value: 0.30637734174377584.
EvaluatorHoldout: Processed 5961 (100.0%) in 4.80 sec. Users per second: 1242


Best trial: 1. Best value: 0.306377:  14%|█▍        | 7/50 [00:34<03:32,  4.93s/it]

[I 2023-07-31 08:55:07,966] Trial 6 finished with value: 0.3061776212894109 and parameters: {'n_items_to_rank': 146}. Best is trial 1 with value: 0.30637734174377584.


In [ ]:
import os
import pandas as pd

# Indirizzo della cartella
folder_path = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10"

# Ottieni tutti i file nella cartella
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Itera su ogni file
for file in files:
    # Costruisci il percorso completo del file
    file_path = os.path.join(folder_path, file)

    # Leggi il file come un DataFrame
    df = pd.read_csv(file_path)  # assume che i tuoi file siano in formato CSV

    # Trova il valore massimo nella colonna 'NDCG'
    max_ndcg_row = df[df['NDCG'] == df['NDCG'].max()]

    n_timesteps = file.

    directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
    os.makedirs(directory_path, exist_ok=True)
    

        filename = directory_path + recommender_instance.RECOMMENDER_NAME + ".csv"

        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)
    

    print(f"File: {file}, NDCG massimo: {max_ndcg}")


In [18]:
import ast
folder_path = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10"
recommenders_instances = [itemKNN_instance, slim_elastic_instance, matrix_fact_instance]
recommenders_instances_test = [itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test]
inference_timesteps = [1,5,25,50]

for inference_timestep in inference_timesteps:
    auto_diffusion_instance_test.inference_timesteps = inference_timestep
    directory_path = os.path.join(folder_path, '_best_timestep', str(inference_timestep))
    os.makedirs(directory_path, exist_ok=True)
    for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
        file_path = os.path.join(folder_path , str(inference_timestep) + auto_diffusion_instance.RECOMMENDER_NAME + '_' + traditional_rec.RECOMMENDER_NAME + '_' + 'TwoLevelRec.csv')
        print(file_path)
        df = pd.read_csv(file_path)  # assume che i tuoi file siano in formato CSV

        # Trova il valore massimo nella colonna 'NDCG'
        max_ndcg_row = df[df['NDCG'] == df['NDCG'].max()]
        print(ast.literal_eval(max_ndcg_row['hyperparams'].iloc[0])['n_items_to_rank'])
        two_level_rec_test = TwoLevelRec(URM_train=URM_train+URM_validation, recommender1= auto_diffusion_instance_test, recommender2=traditional_rec_test, max_cutoff=max(cutoff_list))
        two_level_rec_test.fit(n_items_to_rank= ast.literal_eval(max_ndcg_row['hyperparams'].iloc[0])['n_items_to_rank'])
        result_df, _ =evaluator_test.evaluateRecommender(two_level_rec_test)
        result_df["RECOMMENDER"] = two_level_rec_test.RECOMMENDER_NAME
        result_df["PARAMS"] = str(max_ndcg_row['hyperparams'].iloc[0])
        filename = "best_models_last_" + str(inference_timestep) + "_timesteps" + ".csv"
        filename = os.path.join(directory_path, filename)

        if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        
        

/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10/1DiffusionAutoencoderRecommender_ItemKNNCFRecommender_TwoLevelRec.csv
20
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 5.41 sec. Users per second: 1105
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10/1DiffusionAutoencoderRecommender_SLIMElasticNetRecommender_TwoLevelRec.csv
28
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 4.36 sec. Users per second: 1370
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10/1DiffusionAutoencoderRecommender_IALSRecommender_TwoLevelRec.csv
11
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 3.55 sec. Use

In [17]:
import ast
folder_path = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_50"
recommenders_instances = [itemKNN_instance, slim_elastic_instance, matrix_fact_instance]
recommenders_instances_test = [itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test]
inference_timesteps = [1,5,25,50]

for inference_timestep in inference_timesteps:
    auto_diffusion_instance_test.inference_timesteps = inference_timestep
    directory_path = os.path.join(folder_path, '_best_timestep', str(inference_timestep))
    os.makedirs(directory_path, exist_ok=True)
    for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
        file_path = os.path.join(folder_path , str(inference_timestep) + auto_diffusion_instance.RECOMMENDER_NAME + '_' + traditional_rec.RECOMMENDER_NAME + '_' + 'TwoLevelRec.csv')
        print(file_path)
        df = pd.read_csv(file_path)  # assume che i tuoi file siano in formato CSV

        # Trova il valore massimo nella colonna 'NDCG'
        max_ndcg_row = df[df['NDCG'] == df['NDCG'].max()]
        print(ast.literal_eval(max_ndcg_row['hyperparams'].iloc[0])['n_items_to_rank'])
        two_level_rec_test = TwoLevelRec(URM_train=URM_train+URM_validation, recommender1= auto_diffusion_instance_test, recommender2=traditional_rec_test, max_cutoff=max(cutoff_list))
        two_level_rec_test.fit(n_items_to_rank= ast.literal_eval(max_ndcg_row['hyperparams'].iloc[0])['n_items_to_rank'])
        result_df, _ =evaluator_test.evaluateRecommender(two_level_rec_test)
        result_df["RECOMMENDER"] = two_level_rec_test.RECOMMENDER_NAME
        result_df["PARAMS"] = str(max_ndcg_row['hyperparams'].iloc[0])
        filename = "best_models_last_" + str(inference_timestep) + "_timesteps" + ".csv"
        filename = os.path.join(directory_path, filename)

        if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        
        

/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_50/1DiffusionAutoencoderRecommender_ItemKNNCFRecommender_TwoLevelRec.csv
172
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.


/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Diffusion/DiffusionRecommender.py:266: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:56.)
  user_profile_batch = torch.sparse_csr_tensor(user_profile_batch.indptr,


EvaluatorHoldout: Processed 5976 (100.0%) in 5.64 sec. Users per second: 1060
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_50/1DiffusionAutoencoderRecommender_SLIMElasticNetRecommender_TwoLevelRec.csv
187
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 4.18 sec. Users per second: 1430
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_50/1DiffusionAutoencoderRecommender_IALSRecommender_TwoLevelRec.csv
184
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 3.94 sec. Users per second: 1519
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_50/5DiffusionAutoencoderRecommender_ItemKNNCFRecommender_TwoLevelRec.csv
143
Recommender_Base_Class: URM Detected 194 ( 5.0%) items

In [25]:
import pandas as pd
import os

def csv_to_latex(base_dir):
    cutoff_dirs = ["cutoff_10", "cutoff_50"]
    timesteps_dirs = ["1", "5", "25", "50"]

    latex_output = {}

    for cutoff in cutoff_dirs:
        for timestep in timesteps_dirs:
            path = os.path.join(base_dir, cutoff, timestep, f'best_models_last_{timestep}_timesteps.csv')
            if os.path.exists(path):
                df = pd.read_csv(path)
                latex_str = df.to_latex(index=False)
                output_file_name = os.path.join(base_dir, cutoff, timestep, f'best_models_last_{timestep}_timesteps.tex')
                with open(output_file_name, 'w') as f:
                    f.write(latex_str)
                print(f'Saved LaTeX version to {output_file_name}')
            else:
                print(f'File {path} does not exist.')

base_dir = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50"
csv_to_latex(base_dir)


Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/1/best_models_last_1_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/5/best_models_last_5_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/25/best_models_last_25_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/50/best_models_last_50_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_50/1/best_models_last_1_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_50/5/best_mode

In [3]:
import pandas as pd
import os

def csv_to_latex(base_dir, output_dir):
    # The desired columns
    selected_columns = [
        'NDCG', 
        'PRECISION', 
        'MAP', 
        'NOVELTY', 
        'DIVERSITY_GINI', 
        'DIVERSITY_MEAN_INTER_LIST', 
        'COVERAGE_ITEM', 
        'AVERAGE_POPULARITY'
    ]

    cutoff_dirs = ["cutoff_10", "cutoff_50"]
    timesteps_dirs = ["1", "5", "25", "50"]

    for cutoff in cutoff_dirs:
        for timestep in timesteps_dirs:
            path = os.path.join(base_dir, cutoff, timestep, f'best_models_last_{timestep}_timesteps.csv')
            if os.path.exists(path):
                df = pd.read_csv(path)

                # Filter the dataframe to keep only the desired columns
                df = df[selected_columns]
                latex_str = df.to_latex(index=False)
                
                # Save the file in the specified output directory
                output_file_name = os.path.join(output_dir, f'best_models_{cutoff}_{timestep}_timesteps.tex')
                with open(output_file_name, 'w') as f:
                    f.write(latex_str)
                print(f'Saved LaTeX version to {output_file_name}')
            else:
                print(f'File {path} does not exist.')

base_dir = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50"
output_dir = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/latex_best"
csv_to_latex(base_dir, output_dir)


Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/latex_best/best_models_cutoff_10_1_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/latex_best/best_models_cutoff_10_5_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/latex_best/best_models_cutoff_10_25_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/latex_best/best_models_cutoff_10_50_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/latex_best/best_models_cutoff_50_1_timesteps.tex
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecomm

In [1]:
import pandas as pd
import os

def csv_to_latex(dirs, output_file_path):
    # The desired columns
    selected_columns = [
        'RECOMMENDER',
        'PARAMS',
        'NDCG', 
        'PRECISION', 
        'MAP', 
        'NOVELTY', 
        'DIVERSITY_GINI', 
        'DIVERSITY_MEAN_INTER_LIST', 
        'COVERAGE_ITEM', 
        'COVERAGE_ITEM_HIT', 
        'AVERAGE_POPULARITY'
    ]

    # Column order
    final_column_order = [
        'RECOMMENDER',
        'PARAMS',
        'TIMESTEP',
        'CUTOFF',
        'NDCG', 
        'PRECISION', 
        'MAP', 
        'NOVELTY', 
        'DIVERSITY_GINI', 
        'DIVERSITY_MEAN_INTER_LIST', 
        'COVERAGE_ITEM', 
        'COVERAGE_ITEM_HIT', 
        'AVERAGE_POPULARITY'
    ]

    cutoff_dirs = ["cutoff_10", "cutoff_50"]
    timesteps_dirs = ["1", "5", "25", "50"]

    all_dfs = [] # list to hold all the dataframes

    for base_dir in dirs:
        print(base_dir)
        for cutoff in cutoff_dirs:
            for timestep in timesteps_dirs:
                path = os.path.join(base_dir, cutoff, timestep, f'best_models_last_{timestep}_timesteps.csv')
                if os.path.exists(path):
                    df = pd.read_csv(path)

                    # Filter the dataframe to keep only the desired columns
                    df = df[selected_columns]

                    # Add cutoff, timestep, and directory columns to the dataframe
                    df['CUTOFF'] = cutoff
                    df['TIMESTEP'] = timestep
                    all_dfs.append(df)
    
    # Concatenate all the dataframes into a single one
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df = final_df[final_column_order]  # rearrange columns
    latex_str = final_df.to_latex(index=False)

    # Save the final LaTeX table to the specified output path
    with open(output_file_path, 'w') as f:
        f.write(latex_str)

    print(f'Saved LaTeX version to {output_file_path}')

dirs = [
    "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50",
    "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53"
]
output_file_path = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/final_table.tex"
csv_to_latex(dirs, output_file_path)


/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/final_table.tex


In [51]:
import re

def replace_names_in_latex_file(input_filepath, output_filepath):
    with open(input_filepath, 'r', encoding='utf-8') as file:
        content = file.read()



    replacements = {
        r"ItemKNN\w*": "ItemKNN",
        r"SLIM\w*": "SLIM",
        r"IALS\w*": "IALS",
        r"Diffusi\w*": "Diffusion",
        "PARAMS": "PARAM",
        
    }

    for pattern, replacement in replacements.items():
        content = re.sub(pattern, replacement, content)

    with open(output_filepath, 'w', encoding='utf-8') as file:
        file.write(content)


# Esempio di utilizzo:

input_file = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/final_table.tex"
output_file = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/modified_latex_file.tex"

replace_names_in_latex_file(input_file, output_file)
print(f"Modified content saved to {output_file}")


Modified content saved to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/modified_latex_file.tex


In [54]:
df = pd.read_csv('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10/1/best_models_last_1_timesteps.csv')
for _, row in df.iterrows():
        hybrid_name = row['RECOMMENDER']
        print(hybrid_name)

DiffusionAutoencoderRecommender_ItemKNNCFRecommender_TwoLevelRec
DiffusionAutoencoderRecommender_SLIMElasticNetRecommender_TwoLevelRec
DiffusionAutoencoderRecommender_IALSRecommender_TwoLevelRec


In [65]:
import pandas as pd
import ast
directory = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data"
# Read the CSV
df = pd.read_csv('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53/cutoff_10/1/best_models_last_1_timesteps.csv')

# Define recommenders
recommenders_instances_test = {
    "ItemKNNCFRecommender": itemKNN_instance_test,
    "SLIMElasticNetRecommender": slim_elastic_instance_test,
    "IALSRecommender": matrix_fact_instance_test
}

timesteps = [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 60, 80, 100, 150, 200, 250, 300, 350, 400, 500]
#!!! best model di diffusion autoencoder ha NOISE timesteps =429 quindi non di possono dare INFERENCE timesteps>429 perchè quando viene preso beta o alpha[430+] c'è un errore siccome è stato generato beta solo fino a noise timesteps, quindi 429


# Loop through each row in the dataframe
  # Once you have the model ready, loop through timesteps and perform operations
for timestep in timesteps:  
    auto_diffusion_instance_test.inference_timesteps = timestep
    two_level_rec_test = TwoLevelRec(URM_train=URM_train+URM_validation,
                                        recommender1=auto_diffusion_instance_test,
                                        recommender2=itemKNN_instance_test,
                                        max_cutoff=350)
    for traditional_rec_name, traditional_rec_instance in recommenders_instances_test.items():
        expected_name = auto_diffusion_instance.RECOMMENDER_NAME + '_' + traditional_rec_name + '_TwoLevelRec'
        for _, row in df.iterrows():
            hybrid_name = row['RECOMMENDER']
            if hybrid_name == expected_name:
                # Get the parameters
                params = ast.literal_eval(row['PARAMS'])
                two_level_rec.set_rec2(traditional_rec_instance)
                # Instantiate the TwoLevelRec model with correct parameters
        # Fit the model
        two_level_rec_test.fit(n_items_to_rank=params['n_items_to_rank'])
        result_df, _ =evaluator_test.evaluateRecommender(two_level_rec_test)
        result_df["RECOMMENDER"] = two_level_rec_test.RECOMMENDER_NAME
        result_df["PARAMS"] = str(max_ndcg_row['hyperparams'].iloc[0])
        result_df["TIMESTEPS"] = timestep
        filename = expected_name+ ".csv"
        filename = os.path.join(directory, filename)

        if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

    
        

Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 5.73 sec. Users per second: 1044
EvaluatorHoldout: Processed 5976 (100.0%) in 6.33 sec. Users per second: 944
EvaluatorHoldout: Processed 5976 (100.0%) in 5.73 sec. Users per second: 1043
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 5.98 sec. Users per second: 1000
EvaluatorHoldout: Processed 5976 (100.0%) in 6.44 sec. Users per second: 927
EvaluatorHoldout: Processed 5976 (100.0%) in 5.73 sec. Users per second: 1043
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 6.01 sec. Users per second: 995
EvaluatorHoldout: Processed 5976 (100.0%) in 6.29 sec. Users per second: 950
EvaluatorHoldout: Processed 5976 (100.0%) in 5.74 sec. Users per second: 1042
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interact

IndexError: index 500 is out of bounds for dimension 0 with size 429

In [9]:
import pandas as pd
import ast
directory = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data"
# Read the CSV
df = pd.read_csv('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50/cutoff_10/1/best_models_last_1_timesteps.csv')

# Define recommenders
recommenders_instances_test = {
    #"ItemKNNCFRecommender": itemKNN_instance_test,
    #"SLIMElasticNetRecommender": slim_elastic_instance_test,
    "IALSRecommender": matrix_fact_instance_test
}

timesteps = [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 60, 80, 100, 150, 200, 250, 300, 350, 400, 500]
#!!! best model di diffusion autoencoder ha NOISE timesteps =429 quindi non di possono dare INFERENCE timesteps>429 perchè quando viene preso beta o alpha[430+] c'è un errore siccome è stato generato beta solo fino a noise timesteps, quindi 429

# Loop through each row in the dataframe
  # Once you have the model ready, loop through timesteps and perform operations
for timestep in timesteps:  # Assuming you have a list of timesteps defined somewhere
    auto_diffusion_instance_test.inference_timesteps = timestep
    
                                        
    for traditional_rec_name, traditional_rec_instance in recommenders_instances_test.items():
        expected_name = traditional_rec_name + '_' + auto_diffusion_instance.RECOMMENDER_NAME + '_TwoLevelRec'
        for _, row in df.iterrows():
            hybrid_name = row['RECOMMENDER']
            if hybrid_name == expected_name:
                # Get the parameters
                params = ast.literal_eval(row['PARAMS'])
                # Instantiate the TwoLevelRec model with correct parameters
        # Fit the model
        two_level_rec_test = TwoLevelRec(URM_train=URM_train+URM_validation,
                                        recommender1=itemKNN_instance_test ,
                                        recommender2=auto_diffusion_instance_test,
                                        max_cutoff=350)
        two_level_rec_test.fit(n_items_to_rank=params['n_items_to_rank'])
        result_df, _ =evaluator_test.evaluateRecommender(two_level_rec_test)
        result_df["RECOMMENDER"] = two_level_rec_test.RECOMMENDER_NAME
        result_df["PARAMS"] = params['n_items_to_rank']
        result_df["TIMESTEPS"] = timestep
        filename = expected_name+ ".csv"
        filename = os.path.join(directory, filename)

        if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

    
        

Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 5.15 sec. Users per second: 1160
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 5.84 sec. Users per second: 1023
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 6.63 sec. Users per second: 901
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 7.59 sec. Users per second: 788
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 8.52 sec. Users per second: 702
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.
EvaluatorHoldout: Processed 5976 (100.0%) in 12.74 sec. Users per second: 469
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactio

IndexError: index 500 is out of bounds for dimension 0 with size 429

In [11]:
import pandas as pd
import os

def csv_to_latex(dirs, output_file_path):
    # The desired columns
    selected_columns = [
        'RECOMMENDER',
        'PARAMS',
        'NDCG', 
        'PRECISION', 
        'COVERAGE_ITEM', 
        'COVERAGE_ITEM_HIT', 
        'AVERAGE_POPULARITY'
    ]

    # Column order
    final_column_order = [
        'RECOMMENDER',
        'PARAMS',
        'TIMESTEP',
        'CUTOFF',
        'NDCG', 
        'PRECISION', 
        'COVERAGE_ITEM', 
        'COVERAGE_ITEM_HIT', 
        'AVERAGE_POPULARITY'
    ]

    cutoff_dirs = ["cutoff_10", "cutoff_50"]
    timesteps_dirs = ["1", "5", "25", "50"]

    all_dfs = [] # list to hold all the dataframes

    for base_dir in dirs:
        print(base_dir)
        for cutoff in cutoff_dirs:
            for timestep in timesteps_dirs:
                path = os.path.join(base_dir, cutoff, timestep, f'best_models_last_{timestep}_timesteps.csv')
                if os.path.exists(path):
                    df = pd.read_csv(path)

                    # Filter the dataframe to keep only the desired columns
                    df = df[selected_columns]

                    # Add cutoff, timestep, and directory columns to the dataframe
                    df['CUTOFF'] = cutoff
                    df['TIMESTEP'] = timestep
                    all_dfs.append(df)
    
    # Concatenate all the dataframes into a single one
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df = final_df[final_column_order]  # rearrange columns
    latex_str = final_df.to_latex(index=False)

    # Save the final LaTeX table to the specified output path
    with open(output_file_path, 'w') as f:
        f.write(latex_str)

    print(f'Saved LaTeX version to {output_file_path}')

dirs = [
    "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50",
    "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53"
]
output_file_path = "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/final_table_short.tex"
csv_to_latex(dirs, output_file_path)


/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 09:13:50
/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/2023-07-31 03:41:53
Saved LaTeX version to /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data/FINAL_TABLE/final_table_short.tex
